In [5]:
import cv2
import numpy as np
import gradio as gr

# --- CSS ---
css = """
footer, .label-wrap { display: none !important; }
button[aria-label="Edit"], button[aria-label="Copy"], button[aria-label="Capture from device"] { display: none !important; }
#slider-container { position: relative; padding-bottom: 25px; width: 100% !important; }
#slider-container::before { content: "Smooth"; position: absolute; left: 0; bottom: 5px; font-size: 0.9em; color: #666; font-weight: bold; }
#slider-container::after { content: "Detailed"; position: absolute; right: 0; bottom: 5px; font-size: 0.9em; color: #666; font-weight: bold; }
.gradio-container { max-width: 1100px !important; }
"""

def cartoonize_image_logic(img, intensity):
    if img is None: return None
    
    target_size = 800 
    h, w = img.shape[:2]
    if max(h, w) > target_size:
        scale = target_size / max(h, w)
        img = cv2.resize(img, (int(w * scale), int(h * scale)))

    filter_iterations = int(intensity / 10) + 1
    sigmaColor = int(intensity * 2) + 10
    base_block = int(intensity / 5)
    blockSize = base_block if base_block % 2 == 1 else base_block + 1
    if blockSize < 3: blockSize = 3
    
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img_small = cv2.pyrDown(cv2.pyrDown(img_bgr))
    for _ in range(filter_iterations):
        img_small = cv2.bilateralFilter(img_small, d=9, sigmaColor=sigmaColor, sigmaSpace=7)
    img_cartoon = cv2.pyrUp(cv2.pyrUp(img_small))
    img_cartoon = cv2.resize(img_cartoon, (img_bgr.shape[1], img_bgr.shape[0]))
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.medianBlur(img_gray, 7)
    img_edge = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blockSize, 2)
    img_edge_colored = cv2.cvtColor(img_edge, cv2.COLOR_GRAY2BGR)
    cartoon_bgr = cv2.bitwise_and(img_cartoon, img_edge_colored)
    
    return cv2.cvtColor(cartoon_bgr, cv2.COLOR_BGR2RGB)

with gr.Blocks(css=css) as demo:
    gr.Markdown("# 🎨 影像卡通化轉換器")
    with gr.Row():
        input_img = gr.Image(label="原始影像", type="numpy", sources=["upload"], interactive=True)
        output_img = gr.Image(label="卡通化結果", interactive=False)
    
    with gr.Column(elem_id="slider-container"):
        intensity_slider = gr.Slider(minimum=0, maximum=100, value=50, step=1, label="風格調整")

    intensity_slider.release(cartoonize_image_logic, [input_img, intensity_slider], output_img)
    input_img.change(cartoonize_image_logic, [input_img, intensity_slider], output_img)

demo.close()
demo.launch()

Running on local URL:  http://127.0.0.1:7902

To create a public link, set `share=True` in `launch()`.
